In [268]:
!pip install pandas
!pip install matplotlib
!pip install seaborn


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


**PREDICTING ACCIDENT SEVERITY**

In [2]:
import pickle
import pandas as pd


Reading in Data

In [3]:
accident_chain = pd.read_csv('ACCIDENT_CHAINAGE.csv')
accident_event =  pd.read_csv('ACCIDENT_EVENT.csv')
accident_location = pd.read_csv('ACCIDENT_LOCATION.csv')
accident = pd.read_csv('ACCIDENT.csv')
node_id_complex_int_id = pd.read_csv('NODE_ID_COMPLEX_INT_ID.csv')
node = pd.read_csv('NODE.csv')
person = pd.read_csv('PERSON.csv')
road_surface = pd.read_csv('ROAD_SURFACE_COND.csv')
subdca = pd.read_csv('SUBDCA.csv')
vehicle = pd.read_csv('VEHICLE.csv')

/var/folders/pp/v93sqgpd3dzcpdx34t0c9n280000gn/T/ipykernel_3529/1639358552.py:4: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  accident = pd.read_csv('ACCIDENT.csv')
/var/folders/pp/v93sqgpd3dzcpdx34t0c9n280000gn/T/ipykernel_3529/1639358552.py:7: DtypeWarning: Columns (6,9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  person = pd.read_csv('PERSON.csv')
/var/folders/pp/v93sqgpd3dzcpdx34t0c9n280000gn/T/ipykernel_3529/1639358552.py:10: DtypeWarning: Columns (3,25,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicle = pd.read_csv('VEHICLE.csv')


In [4]:
accident

,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,6,0,1,0,5,1,1,Cross intersection,3,60
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,4,0,1,0,3,1,2,T intersection,3,70
2,T20060000022,14/01/2006,12:10:00,7,Fall from or in moving vehicle,7,Saturday,190,FELL IN/FROM VEHICLE,MEL,...,2,1,0,0,1,1,5,Not at intersection,2,100
3,T20060000023,14/01/2006,11:49:00,1,Collision with vehicle,7,Saturday,130,REAR END(VEHICLES IN SAME LANE),MEL,...,2,1,0,0,1,1,2,T intersection,2,80
4,T20060000026,14/01/2006,10:45:00,1,Collision with vehicle,7,Saturday,121,RIGHT THROUGH,MEL,...,3,0,3,0,0,1,5,Not at intersection,3,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203703,T20200019239,1/11/2020,12:11:00,1,Collision with vehicle,0,Sunday,142,LEAVING PARKING,MEL,...,4,1,0,0,3,1,5,Not at intersection,2,50
203704,T20200019247,1/11/2020,15:30:00,4,Collision with a fixed object,1,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...,MEL,...,2,2,0,0,0,1,5,Not at intersection,2,999
203705,T20200019250,1/11/2020,18:00:00,1,Collision with vehicle,0,Sunday,116,LEFT NEAR (INTERSECTIONS ONLY),MEL,...,2,1,0,0,1,1,1,Cross intersection,2,60
203706,T20200019253,1/11/2020,12:00:00,6,Vehicle overturned (no collision),1,Sunday,180,OFF CARRIAGEWAY ON RIGHT BEND,VCD,...,1,1,0,0,0,1,5,Not at intersection,2,80


In [5]:
accident.columns

Index(['ACCIDENT_NO', 'ACCIDENTDATE', 'ACCIDENTTIME', 'ACCIDENT_TYPE',
       'Accident Type Desc', 'DAY_OF_WEEK', 'Day Week Description', 'DCA_CODE',
       'DCA Description', 'DIRECTORY', 'EDITION', 'PAGE', 'GRID_REFERENCE_X',
       'GRID_REFERENCE_Y', 'LIGHT_CONDITION', 'Light Condition Desc',
       'NODE_ID', 'NO_OF_VEHICLES', 'NO_PERSONS', 'NO_PERSONS_INJ_2',
       'NO_PERSONS_INJ_3', 'NO_PERSONS_KILLED', 'NO_PERSONS_NOT_INJ',
       'POLICE_ATTEND', 'ROAD_GEOMETRY', 'Road Geometry Desc', 'SEVERITY',
       'SPEED_ZONE'],
      dtype='object')

Only interested in Factors we can control and know before the crash e.g. Speed important but type of Accident not as it is only known after the fact

In [6]:
accident['SPEED_ZONE'].value_counts()

SPEED_ZONE
60     69133
50     36149
100    31240
80     27794
70     15145
999    10709
40      8937
110     2151
90       940
888      930
30       269
777      249
75        62
Name: count, dtype: int64

In [7]:
accident['Day Week Description'].value_counts()

Day Week Description
Friday       32240
Thursday     30950
Wednesday    29939
Tuesday      29132
Saturday     28883
Monday       27383
Sunday       25181
Name: count, dtype: int64

In [8]:
accident['Light Condition Desc'].value_counts()

Light Condition Desc
Day                           136360
Dark Street lights on          33105
Dusk/Dawn                      16485
Dark No street lights          11490
Unknown                         3689
Dark Street lights unknown      2063
Dark Street lights off           516
Name: count, dtype: int64

In [9]:
accident['Road Geometry Desc'].value_counts()

Road Geometry Desc
Not at intersection      103274
T intersection            47908
Cross intersection        47133
Multiple intersection      4196
Y intersection              595
Unknown                     433
Dead end                    155
Private property              9
Road closure                  5
Name: count, dtype: int64

In [10]:
accident['SEVERITY'].value_counts()

SEVERITY
3    131561
2     68436
1      3707
4         4
Name: count, dtype: int64

In [11]:
# 1 is most severe and 4 is least severt
accident.loc[accident['SEVERITY']==1]

,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
52,T20060000313,22/01/2006,23:30:00,4,Collision with a fixed object,1,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...,MEL,...,1,0,0,1,0,1,5,Not at intersection,1,90
54,T20060000317,1/01/2006,06:30:00,4,Collision with a fixed object,1,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...,MEL,...,5,3,0,2,0,1,5,Not at intersection,1,70
59,T20060000345,3/01/2006,12:50:00,4,Collision with a fixed object,3,Tuesday,173,RIGHT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHIC...,VCS,...,1,0,0,1,0,1,5,Not at intersection,1,110
78,T20060000438,25/01/2006,03:50:00,1,Collision with vehicle,4,Wednesday,160,VEHICLE COLLIDES WITH VEHICLE PARKED ON LEFT O...,MEL,...,4,1,0,2,1,1,5,Not at intersection,1,90
94,T20060000508,2/01/2006,16:10:00,4,Collision with a fixed object,2,Monday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...,VCS,...,2,1,0,1,0,1,5,Not at intersection,1,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203589,T20200017372,30/09/2020,06:49:00,4,Collision with a fixed object,4,Wednesday,173,RIGHT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHIC...,MEL,...,1,0,0,1,0,1,5,Not at intersection,1,80
203641,T20200018096,14/10/2020,00:15:00,4,Collision with a fixed object,4,Wednesday,183,OFF LEFT BEND INTO OBJECT/PARKED VEHICLE ...,VCD,...,1,0,0,1,0,1,5,Not at intersection,1,100
203659,T20200018319,17/10/2020,17:19:00,4,Collision with a fixed object,7,Saturday,173,RIGHT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHIC...,MEL,...,1,0,0,1,0,1,5,Not at intersection,1,100
203670,T20200018424,19/10/2020,16:22:00,1,Collision with vehicle,2,Monday,119,OTHER ADJACENT (INTERSECTIONS ONLY),MEL,...,2,0,1,1,0,1,2,T intersection,1,100


In [12]:
#speed will be a determining factor so for all speeds greater than 110, remove the row
accident_filtered = accident[accident['SPEED_ZONE'] <= 110]


In [13]:
type(person['SEATING_POSITION'][3]),person['SEATING_POSITION'][3]

(str, 'D ')

In [14]:
person.columns

Index(['ACCIDENT_NO', 'PERSON_ID', 'VEHICLE_ID', 'SEX', 'AGE', 'Age Group',
       'INJ_LEVEL', 'Inj Level Desc', 'SEATING_POSITION', 'HELMET_BELT_WORN',
       'ROAD_USER_TYPE', 'Road User Type Desc', 'LICENCE_STATE',
       'PEDEST_MOVEMENT', 'POSTCODE', 'TAKEN_HOSPITAL', 'EJECTED_CODE'],
      dtype='object')

In [15]:
#interested in the driver as we will see if age/sex have any impact on severity of collission
drivers = person[person['SEATING_POSITION'] == 'D ']
drivers = drivers[['ACCIDENT_NO','SEX','AGE','HELMET_BELT_WORN']]

In [16]:
drivers['HELMET_BELT_WORN'].value_counts()

HELMET_BELT_WORN
1    110756
1     94892
9     48396
9     47628
6     20101
6     17917
2      3370
2      2652
8      1668
7      1199
7      1013
8       732
5       487
5       417
4         1
Name: count, dtype: int64

In [17]:
drivers['HELMET_BELT_WORN'] = drivers['HELMET_BELT_WORN'].astype(int)

In [18]:
drivers['HELMET_BELT_WORN'].value_counts()

HELMET_BELT_WORN
1    205648
9     96024
6     38018
2      6022
8      2400
7      2212
5       904
4         1
Name: count, dtype: int64

In [19]:
drivers

,ACCIDENT_NO,SEX,AGE,HELMET_BELT_WORN
3,T20060000010,M,72.0,1
4,T20060000010,F,62.0,1
5,T20060000010,M,39.0,1
8,T20060000018,M,30.0,1
9,T20060000018,F,20.0,1
...,...,...,...,...
490943,T20200019247,M,21.0,1
490944,T20200019250,M,45.0,1
490945,T20200019250,M,56.0,6
490946,T20200019253,M,65.0,6


In [20]:
road_surface

,ACCIDENT_NO,SURFACE_COND,Surface Cond Desc,SURFACE_COND_SEQ
0,T20060000010,1,Dry,1
1,T20060000018,1,Dry,1
2,T20060000022,1,Dry,1
3,T20060000023,1,Dry,1
4,T20060000026,1,Dry,1
...,...,...,...,...
205025,T20200019239,1,Dry,0
205026,T20200019247,1,Dry,1
205027,T20200019250,1,Dry,0
205028,T20200019253,1,Dry,1


In [21]:
road_surface['Surface Cond Desc'].value_counts()

Surface Cond Desc
Dry        160269
Wet         32261
Unknown     10416
Muddy        1257
Icy           689
Snowy         138
Name: count, dtype: int64

In [22]:
vehicle.columns

Index(['ACCIDENT_NO', 'VEHICLE_ID', 'VEHICLE_YEAR_MANUF', 'VEHICLE_DCA_CODE',
       'INITIAL_DIRECTION', 'ROAD_SURFACE_TYPE', 'Road Surface Type Desc',
       'REG_STATE', 'VEHICLE_BODY_STYLE', 'VEHICLE_MAKE', 'VEHICLE_MODEL',
       'VEHICLE_POWER', 'VEHICLE_TYPE', 'Vehicle Type Desc', 'VEHICLE_WEIGHT',
       'CONSTRUCTION_TYPE', 'FUEL_TYPE', 'NO_OF_WHEELS', 'NO_OF_CYLINDERS',
       'SEATING_CAPACITY', 'TARE_WEIGHT', 'TOTAL_NO_OCCUPANTS',
       'CARRY_CAPACITY', 'CUBIC_CAPACITY', 'FINAL_DIRECTION', 'DRIVER_INTENT',
       'VEHICLE_MOVEMENT', 'TRAILER_TYPE', 'VEHICLE_COLOUR_1',
       'VEHICLE_COLOUR_2', 'CAUGHT_FIRE', 'INITIAL_IMPACT', 'LAMPS',
       'LEVEL_OF_DAMAGE', 'OWNER_POSTCODE', 'TOWED_AWAY_FLAG',
       'TRAFFIC_CONTROL', 'Traffic Control Desc'],
      dtype='object')

In [31]:
vehicle['VEHICLE_YEAR_MANUF'].value_counts()

easiest_date = vehicle['VEHICLE_YEAR_MANUF'].min()
latest_date = vehicle['VEHICLE_YEAR_MANUF'].max()

# Create a Series with the extracted dates
manufactured_dates = pd.Series({
    'Earliest Date Manufactured': easiest_date,
    'Latest Date Manufactured': latest_date
})
manufactured_dates.name = "Newest and Oldest Vehicles"
manufactured_dates_df = manufactured_dates.to_frame()

# Print the DataFrame with a nice format
print(manufactured_dates_df)


                            Newest and Oldest Vehicles
Earliest Date Manufactured                         0.0
Latest Date Manufactured                        3001.0


In [352]:
# only keep cars manufactured between 1890 and 2020
vehicle_filtered = vehicle[(vehicle['VEHICLE_YEAR_MANUF'] >= 1890) & (vehicle['VEHICLE_YEAR_MANUF'] <= 2020)]
#turn year manufactured to be years old (years since 2020)
vehicle_filtered['VEHICLE_YEARS_OLD'] = 2020 - vehicle_filtered['VEHICLE_YEAR_MANUF']

/var/folders/pp/v93sqgpd3dzcpdx34t0c9n280000gn/T/ipykernel_9094/2948637117.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_filtered['VEHICLE_YEARS_OLD'] = 2020 - vehicle_filtered['VEHICLE_YEAR_MANUF']


In [353]:
vehicle_filtered

,ACCIDENT_NO,VEHICLE_ID,VEHICLE_YEAR_MANUF,VEHICLE_DCA_CODE,INITIAL_DIRECTION,ROAD_SURFACE_TYPE,Road Surface Type Desc,REG_STATE,VEHICLE_BODY_STYLE,VEHICLE_MAKE,...,VEHICLE_COLOUR_2,CAUGHT_FIRE,INITIAL_IMPACT,LAMPS,LEVEL_OF_DAMAGE,OWNER_POSTCODE,TOWED_AWAY_FLAG,TRAFFIC_CONTROL,Traffic Control Desc,VEHICLE_YEARS_OLD
0,T20060000010,A,1996.0,2,SW,1,Paved,V,SEDAN,MITSUB,...,ZZ,2,F,2,3,3130.0,1,1,Stop-go lights,24.0
1,T20060000010,B,2003.0,1,NW,1,Paved,V,COUPE,UNKN,...,ZZ,2,2,2,3,3977.0,1,1,Stop-go lights,17.0
2,T20060000010,C,2001.0,8,NW,1,Paved,V,SEDAN,FORD,...,ZZ,2,F,2,2,3804.0,2,1,Stop-go lights,19.0
3,T20060000018,A,1998.0,2,S,1,Paved,V,DC UTE,TOYOTA,...,ZZ,2,9,0,3,3175.0,1,0,No control,22.0
4,T20060000018,B,1991.0,1,N,1,Paved,V,SEDAN,SUBARU,...,ZZ,2,F,0,5,3805.0,1,11,Giveway sign,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365237,T20200019247,A,2006.0,1,S,3,Gravel,V,SEDAN,VOLKS,...,ZZ,2,F,2,5,3224.0,1,0,No control,14.0
365238,T20200019250,A,2007.0,1,NE,1,Paved,V,SEDAN,VOLKS,...,ZZ,2,1,2,1,3072.0,2,11,Giveway sign,13.0
365239,T20200019250,B,2007.0,2,NW,1,Paved,V,SOLO,YAMAHA,...,ZZ,2,5,2,1,3082.0,2,0,No control,13.0
365240,T20200019253,A,2007.0,1,SW,1,Paved,V,SOLO,K T M,...,ZZ,2,5,2,1,3444.0,2,0,No control,13.0


In [354]:
# Assuming Lamps are number of working headlights
vehicle_filtered['LAMPS'].value_counts()
vehicle_filtered.loc[vehicle_filtered['LAMPS']==1]

,ACCIDENT_NO,VEHICLE_ID,VEHICLE_YEAR_MANUF,VEHICLE_DCA_CODE,INITIAL_DIRECTION,ROAD_SURFACE_TYPE,Road Surface Type Desc,REG_STATE,VEHICLE_BODY_STYLE,VEHICLE_MAKE,...,VEHICLE_COLOUR_2,CAUGHT_FIRE,INITIAL_IMPACT,LAMPS,LEVEL_OF_DAMAGE,OWNER_POSTCODE,TOWED_AWAY_FLAG,TRAFFIC_CONTROL,Traffic Control Desc,VEHICLE_YEARS_OLD
13,T20060000042,A,2004.0,1,SE,1,Paved,V,WAGON,TOYOTA,...,ZZ,2,9,1,3,3910.0,1,0,No control,16.0
30,T20060000111,A,2000.0,1,W,1,Paved,V,SEDAN,MITSUB,...,ZZ,2,F,1,5,3150.0,1,1,Stop-go lights,20.0
31,T20060000111,B,1990.0,2,E,1,Paved,V,COUPE,HONDA,...,ZZ,2,1,1,5,3802.0,1,1,Stop-go lights,30.0
47,T20060000141,B,1985.0,2,E,1,Paved,V,S WAG,L ROV,...,ZZ,2,1,1,3,3840.0,1,0,No control,35.0
75,T20060000270,A,1998.0,1,SE,1,Paved,V,SEDAN,UNKN,...,ZZ,2,F,1,4,3781.0,1,0,No control,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365094,T20200017750,A,2019.0,1,E,1,Paved,V,SCOOTR,HONDA,...,ZZ,2,5,1,2,4007.0,9,1,Stop-go lights,1.0
365112,T20200017909,A,2004.0,1,SE,1,Paved,V,UTIL,FORD,...,ZZ,2,F,1,5,3500.0,1,0,No control,16.0
365131,T20200018088,C,2018.0,1,E,1,Paved,V,SEDAN,MITSUB,...,ZZ,2,5,1,3,2000.0,1,0,No control,2.0
365167,T20200018359,A,2015.0,1,W,1,Paved,V,SOLO,YAMAHA,...,ZZ,2,F,1,5,3028.0,1,0,No control,5.0


In [355]:
accident_filtered.columns

Index(['ACCIDENT_NO', 'ACCIDENTDATE', 'ACCIDENTTIME', 'ACCIDENT_TYPE',
       'Accident Type Desc', 'DAY_OF_WEEK', 'Day Week Description', 'DCA_CODE',
       'DCA Description', 'DIRECTORY', 'EDITION', 'PAGE', 'GRID_REFERENCE_X',
       'GRID_REFERENCE_Y', 'LIGHT_CONDITION', 'Light Condition Desc',
       'NODE_ID', 'NO_OF_VEHICLES', 'NO_PERSONS', 'NO_PERSONS_INJ_2',
       'NO_PERSONS_INJ_3', 'NO_PERSONS_KILLED', 'NO_PERSONS_NOT_INJ',
       'POLICE_ATTEND', 'ROAD_GEOMETRY', 'Road Geometry Desc', 'SEVERITY',
       'SPEED_ZONE'],
      dtype='object')

Before Merging All DataFrames on ACCIDENT_NO, Remove All irrelevant columns to reduce dimensionality of data 

In [356]:
accident_filtered = accident_filtered[['ACCIDENT_NO','ACCIDENTTIME','DAY_OF_WEEK','Day Week Description','LIGHT_CONDITION','Light Condition Desc','ROAD_GEOMETRY','Road Geometry Desc','SEVERITY','SPEED_ZONE']]

In [357]:
road_surface.columns

Index(['ACCIDENT_NO', 'SURFACE_COND', 'Surface Cond Desc', 'SURFACE_COND_SEQ'], dtype='object')

In [358]:
road_surface = road_surface[['ACCIDENT_NO','SURFACE_COND','Surface Cond Desc']]

In [359]:
vehicle_filtered.columns

Index(['ACCIDENT_NO', 'VEHICLE_ID', 'VEHICLE_YEAR_MANUF', 'VEHICLE_DCA_CODE',
       'INITIAL_DIRECTION', 'ROAD_SURFACE_TYPE', 'Road Surface Type Desc',
       'REG_STATE', 'VEHICLE_BODY_STYLE', 'VEHICLE_MAKE', 'VEHICLE_MODEL',
       'VEHICLE_POWER', 'VEHICLE_TYPE', 'Vehicle Type Desc', 'VEHICLE_WEIGHT',
       'CONSTRUCTION_TYPE', 'FUEL_TYPE', 'NO_OF_WHEELS', 'NO_OF_CYLINDERS',
       'SEATING_CAPACITY', 'TARE_WEIGHT', 'TOTAL_NO_OCCUPANTS',
       'CARRY_CAPACITY', 'CUBIC_CAPACITY', 'FINAL_DIRECTION', 'DRIVER_INTENT',
       'VEHICLE_MOVEMENT', 'TRAILER_TYPE', 'VEHICLE_COLOUR_1',
       'VEHICLE_COLOUR_2', 'CAUGHT_FIRE', 'INITIAL_IMPACT', 'LAMPS',
       'LEVEL_OF_DAMAGE', 'OWNER_POSTCODE', 'TOWED_AWAY_FLAG',
       'TRAFFIC_CONTROL', 'Traffic Control Desc', 'VEHICLE_YEARS_OLD'],
      dtype='object')

In [361]:
vehicle_filtered = vehicle_filtered[['ACCIDENT_NO','VEHICLE_BODY_STYLE','TOTAL_NO_OCCUPANTS','VEHICLE_YEARS_OLD', 'LAMPS']]

In [362]:
vehicle_filtered['TOTAL_NO_OCCUPANTS'].value_counts()

TOTAL_NO_OCCUPANTS
1.0     244402
2.0      49342
3.0      13584
0.0       9338
4.0       6131
5.0       2321
6.0        418
7.0        136
8.0         70
9.0         41
12.0        22
10.0        19
13.0        15
11.0        14
20.0        12
15.0         6
47.0         4
16.0         4
14.0         4
17.0         4
25.0         3
19.0         3
18.0         3
30.0         3
21.0         3
27.0         2
48.0         2
22.0         2
31.0         2
44.0         1
34.0         1
35.0         1
32.0         1
33.0         1
28.0         1
29.0         1
37.0         1
49.0         1
24.0         1
Name: count, dtype: int64

In [363]:
vehicle_filtered.loc[vehicle_filtered['TOTAL_NO_OCCUPANTS']>20]

,ACCIDENT_NO,VEHICLE_BODY_STYLE,TOTAL_NO_OCCUPANTS,VEHICLE_YEARS_OLD,LAMPS
7591,T20060015893,BUS,47.0,40.0,2
18519,T20060038536,BUS,21.0,38.0,0
21870,T20060045093,BUS,35.0,30.0,1
26732,T20070006026,BUS,48.0,23.0,0
37584,T20070027732,BUS,30.0,19.0,2
60242,T20080024078,BUS,44.0,22.0,2
74771,T20090004229,BUS,34.0,26.0,0
85059,T20090025716,BUS,27.0,14.0,1
88002,T20090032198,BUS,47.0,18.0,9
95225,T20090047957,VAN,21.0,19.0,2


In [364]:
#Going to Merge and Have one row Per Driver, this will have multiple rows per Incident

merged_data = drivers.merge(accident_filtered, on='ACCIDENT_NO', how='inner').merge(road_surface, on='ACCIDENT_NO', how='inner').merge(vehicle_filtered, on='ACCIDENT_NO', how='inner')
merged_data

,ACCIDENT_NO,SEX,AGE,HELMET_BELT_WORN,ACCIDENTTIME,DAY_OF_WEEK,Day Week Description,LIGHT_CONDITION,Light Condition Desc,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE,SURFACE_COND,Surface Cond Desc,VEHICLE_BODY_STYLE,TOTAL_NO_OCCUPANTS,VEHICLE_YEARS_OLD,LAMPS
0,T20060000010,M,72.0,1,12:42:00,6,Friday,1,Day,1,Cross intersection,3,60,1,Dry,SEDAN,2.0,24.0,2
1,T20060000010,M,72.0,1,12:42:00,6,Friday,1,Day,1,Cross intersection,3,60,1,Dry,COUPE,1.0,17.0,2
2,T20060000010,M,72.0,1,12:42:00,6,Friday,1,Day,1,Cross intersection,3,60,1,Dry,SEDAN,3.0,19.0,2
3,T20060000010,F,62.0,1,12:42:00,6,Friday,1,Day,1,Cross intersection,3,60,1,Dry,SEDAN,2.0,24.0,2
4,T20060000010,F,62.0,1,12:42:00,6,Friday,1,Day,1,Cross intersection,3,60,1,Dry,COUPE,1.0,17.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634032,T20200019250,M,45.0,1,18:00:00,0,Sunday,1,Day,1,Cross intersection,2,60,1,Dry,SOLO,1.0,13.0,2
634033,T20200019250,M,56.0,6,18:00:00,0,Sunday,1,Day,1,Cross intersection,2,60,1,Dry,SEDAN,1.0,13.0,2
634034,T20200019250,M,56.0,6,18:00:00,0,Sunday,1,Day,1,Cross intersection,2,60,1,Dry,SOLO,1.0,13.0,2
634035,T20200019253,M,65.0,6,12:00:00,1,Sunday,1,Day,5,Not at intersection,2,80,1,Dry,SOLO,1.0,13.0,2


In [365]:
merged_data.columns

Index(['ACCIDENT_NO', 'SEX', 'AGE', 'HELMET_BELT_WORN', 'ACCIDENTTIME',
       'DAY_OF_WEEK', 'Day Week Description', 'LIGHT_CONDITION',
       'Light Condition Desc', 'ROAD_GEOMETRY', 'Road Geometry Desc',
       'SEVERITY', 'SPEED_ZONE', 'SURFACE_COND', 'Surface Cond Desc',
       'VEHICLE_BODY_STYLE', 'TOTAL_NO_OCCUPANTS', 'VEHICLE_YEARS_OLD',
       'LAMPS'],
      dtype='object')

In [366]:
merged_data = merged_data[merged_data['AGE'] > 0]

In [368]:
merged_data['LAMPS'].value_counts()

LAMPS
2    477553
1     61263
0     44713
9     35173
Name: count, dtype: int64

In [375]:
merged_data.to_pickle('crash_processed_data.pkl')


In [370]:
drivers['HELMET_BELT_WORN'].value_counts()

HELMET_BELT_WORN
1    205648
9     96024
6     38018
2      6022
8      2400
7      2212
5       904
4         1
Name: count, dtype: int64

In [374]:
vehicle.loc[vehicle['LAMPS']==9,'VEHICLE_BODY_STYLE'].value_counts()

VEHICLE_BODY_STYLE
SEDAN     10725
           6799
WAGON      1979
SOLO       1841
S WAG      1433
          ...  
MACHNE        1
TRICAR        1
TRA           1
OTH           1
TRITRA        1
Name: count, Length: 85, dtype: int64